In [66]:
import json
import requests
from src import utils
import time
import os

In [67]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [68]:
api = utils.read_json_file(api_path)

In [69]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [70]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [71]:
output_file_name = 'one-shot-cot-public_mwp_data_v2_test.jsonl'

In [72]:
# os.makedirs(output_path, exist_ok=True)
# output_file_name = 'result-' + input_file.split('/')[-1]
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/one-shot-cot-public_mwp_data_v2_test.jsonl


In [73]:
# jsonl_writer = utils.JSONLWriter(output_file)

In [74]:
data = utils.read_jsonlines(input_file)

In [75]:
len(data)

3750

In [76]:
# api

In [77]:
# url = api['url']

In [78]:
# headers = api['headers']

In [79]:
# question = "질문 : 파운드 스털링은 가치가 있나요? 용어설명 :  사실목록 : ['1파운드는 법정 화폐입니다.', '법정 화폐는 정부 법령에 의해 뒷받침되며 내재적 가치가 없습니다', '2020년 5월 기준 1파운드 스털링은 약 1.24달러의 가치가 있습니다.']. 사실목록을 바탕으로 질문에 대한 답변을 사실 또는 거짓으로 대답하세요"/

In [80]:
# headers

In [81]:
PROMPT1 = "질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? \
풀이: 동그란 단추는 6개, 네모난 단추는 3개, 세모난 단추는 3개가 있습니다. 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 이를 수식으로 나타내면, 동그란 단추의 개수(6개) - 네모난 단추의 개수(3개) = 3개입니다. 따라서 동그란 단추는 네모난 단추보다 3개가 더 많습니다. \
정답: 3 \\n"


In [82]:
# data[1]

In [83]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [84]:
model = model_api(api['url'], api['headers'])

In [85]:
BODY_TEMP = api['body']

In [86]:
# body = json.dumps(body_temp)

In [87]:
# print(body)

In [88]:
# response = api.generate(body)

In [89]:
# context = response.json()['data'][0]['result'][0]

In [90]:
# print(context)

In [91]:
def get_mwp_one_shot_cot(pre_prompt, que):
    suf_prompt = f'질문: {que} 풀이 : '
    prompt = pre_prompt + suf_prompt
    return prompt

In [92]:
def get_mwp_zs_cot(que):
    prompt = f"질문: {que} 정답: 질문을 단계적으로 풀어보면, "
    return prompt

In [93]:
def get_mwp_zs(que):
    prompt = f'질문: {que} 정답 : '
    return prompt

In [94]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [95]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [96]:
cnt = 0
split_cnt = 0
for ins in data:

    kor_que = ins['question']
    
    prompt = get_gsm8k_one_shot_cot(PROMPT1, kor_que)
    
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('------------')
    print(f"answer:\n{ins['answer']}")
    print('============')
#     ins['zs_infer'] = res_context
#     jsonl_writer.write_json_line(ins)

    if cnt == 10:
        break
    else:
        cnt += 1
    


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? 풀이: 동그란 단추는 6개, 네모난 단추는 3개, 세모난 단추는 3개가 있습니다. 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 이를 수식으로 나타내면, 동그란 단추의 개수(6개) - 네모난 단추의 개수(3개) = 3개입니다. 따라서 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 정답: 3 \n질문: 지영이네 반 친구들이 키가 작은 순서대로 줄을 서있습니다. 지영이는 앞에서부터 6번째에 줄을 섰으며, 지영이네 반은 모두 18명입니다. 키가 큰 사람부터 다시 줄을 세운다면, 지영이는 앞에서부터 몇 번째로 줄을 서게 될까요? 풀이 : 
res_context : 
6번째
------------
answer:
13


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? 풀이: 동그란 단추는 6개, 네모난 단추는 3개, 세모난 단추는 3개가 있습니다. 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 이를 수식으로 나타내면, 동그란 단추의 개수(6개) - 네모난 단추의 개수(3개) = 3개입니다. 따라서 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 정답: 3 \n질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? 풀이 : 
res_context : 
동그란 단추는 6개, 네모난 단추는 3개, 세모난 단추는 3개가 있습니다. 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 이를 수식으로 나타내면, 동그란 단추의 개수(6개) - 네모난 단추의 개수(3개) = 3개입니다. 따라서 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 정답: 3
------------
answer:
3


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

prompt : 
질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? 풀이: 동그란 단추는 6개, 네모난 단추는 3개, 세모난 단추는 3개가 있습니다. 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 이를 수식으로 나타내면, 동그란 단추의 개수(6개) - 네모난 단추의 개수(3개) = 3개입니다. 따라서 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 정답: 3 \n질문: 은경이는 15일마다, 세찬이는 18일마다 가족 농장에 간다고 합니다. 8월 1일에 두 사람이 함께 가족 농장에 갔다면 다음번에 함께 가족 농장에 가는 날을 구하고, 월과 일을 더해보시오. 풀이 : 
res_context : 
두 사람이 함께 가족 농장에 가는 날은 다음과 같습니다. - 8월 1일 - 세찬이와 은경이가 15일마다 함께 가족 농장에 갑니다. - 8월 2일 - 세찬이와 은경이가 18일마다 함께 가족 농장에 갑니다. - 8월 3일 - 세찬이와 은경이가 20일마다 함께 가족 농장에 갑니다. - 8월 4일 - 세찬이와 은경이가 24일마다 함께 가족 농장에 갑니다. - 8월 5일 - 세찬이와 은경이가 28일마다 함께 가족 농장에 갑니다. - 8월 6일 - 세찬이와 은경이가 30일마다 함께 가족 농장에 갑니다. - 8월 7일 - 세찬이와 은경이가 31일마다 함께 가족 농장에 갑니다. - 8월 8일 - 세찬이와 은경이가 36일마다 함께 가족 농장에 갑니다. - 8월 9일 - 세찬이와 은경이가 37일마다 함께 가족 농장에 갑니다. - 8월 10일 - 세찬이와 은경이가 38일마다 함께 가족 농장에 갑니다. - 8월 11일 - 세찬이와 은경이가 39일마다 함께 가족 농장에 갑니다. - 8월 12일 - 세찬이와 은경이가 40일마다 함께 가족 농장에 갑니다. - 8월 13일 - 세찬이와 은경이가 41일마다 함께 가족 농장에 갑니다. - 8월 14

/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? 풀이: 동그란 단추는 6개, 네모난 단추는 3개, 세모난 단추는 3개가 있습니다. 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 이를 수식으로 나타내면, 동그란 단추의 개수(6개) - 네모난 단추의 개수(3개) = 3개입니다. 따라서 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 정답: 3 \n질문: 어제는 1cm 3mm, 오늘은 2cm 4mm의 비가 내렸습니다. 어제와 오늘 내린 비의 양은 모두 몇 mm일까요? 풀이 : 
res_context : 
1cm 3mm와 2cm 4mm는 서로 다릅니다. 따라서 어제와 오늘 내린 비의 양은 서로 다릅니다. 정답: 2cm 4mm
------------
answer:
37


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 반짇고리 안에는 여러 가지 모양의 단추가 있습니다. 동그란 단추는 6개가 들어있고, 네모난 단추와 세모난 단추는 각각 3개씩 들어 있습니다. 동그란 단추는 네모난 단추보다 몇 개가 더 많습니까? 풀이: 동그란 단추는 6개, 네모난 단추는 3개, 세모난 단추는 3개가 있습니다. 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 이를 수식으로 나타내면, 동그란 단추의 개수(6개) - 네모난 단추의 개수(3개) = 3개입니다. 따라서 동그란 단추는 네모난 단추보다 3개가 더 많습니다. 정답: 3 \n질문: 귤을 8개의 바구니에 똑같이 나누어 담으려고 합니다. 그러나 잘못 하여 12개씩 7개의 바구니에 넣어 4개의 귤이 남았다고 합니다. 바르게 나누었을 때 한 바구니에 몇 개의 귤을 담게 되는지 구하시오. 풀이 : 
res_context : 
주어진 상황에서 귤을 8개의 바구니에 똑같이 나누어 담는다고 했으므로, 귤을 8개의 바구니에 담을 수 있습니다. 따라서 한 바구니에 8개의 귤을 담게 됩니다. 정답: 8
------------
answer:
11
